## Оценка популярности продуктов магазина компьютерных игр. Планирование рекламных компаний.

### Описание проекта
#### Вы работаете в интернет-магазине «Стримчик», который продаёт по всему миру компьютерные игры. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). 
#### Задача - выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.
#### Рассмотрены данные до 2016 года. Отработаны принципы работы с данными. Спрогнозированы продажи на 2017 год по данным 2016.

In [ ]:
import pandas as pd #импортировал библиотеку pandas
import matplotlib.pyplot as plt #импортировал библиотеку mathplotlib
import seaborn as sns #импортировал библиотеку seaborn
import numpy as np #импортировал библиотеку numpy
from scipy import stats as st # из библиотеки scipy импортировал модуль stats

## Шаг 1. Ознакомление с данными.

In [ ]:
try:
    df = pd.read_csv(r"D:\DOCS\datasets\games.csv")
except FileNotFoundError:
    df = pd.read_csv('/datasets/games.csv')

In [ ]:
# Вывел первые 20 строк DataFrame
df.head(20)

In [ ]:
#получил основную информацию о датафрейме
df.info(); 

In [ ]:
#получил количество пропусков в каждом столбце
df.isna().sum() 

In [ ]:
# вывел для наглядности пропущенные значения бары
def pass_value_barh(df):
    try:
        (
            (df.isna().mean()*100)
            .to_frame()
            .rename(columns = {0:'space'})
            .query('space > 0')
            .sort_values(by = 'space', ascending = True)
            .plot(kind = 'barh', figsize = (19,6), rot = -5, legend = False, fontsize = 16)
            .set_title('Количество пропусков' + "\n", fontsize = 22, color = 'SteelBlue')    
        );    
    except:
        print('пропусков не осталось :) ')

In [ ]:
pass_value_barh(df)

##### В столбцах Genre  и Name есть небольшое количество пропусков. Эти строки планируется удалить. Также поступим со строками в коропых есть пропуски в столбце Year_of_Release. Пропуски в значениях рейтинга заменим на значение unknown. Пропуски в столбцах Critic_Score и User_Score решено оставить с свзи с тем, что заполнение их медианным и средним значением отразилось бы на дальнейшем исследовании и могло бы привести к некорректным результатам.

## Шаг 2. Предобработка данных

In [ ]:
# Удалил строки с пропущенным значением в названии игры
df.dropna(subset=['Name'], inplace = True)

##### Пропуски в столбце Name восстановить не получится. Так как данных строк небольшое количество, их удаление не повлияет на дальнейший анализ. В связи с этим, решил удалить эти строки

In [ ]:
# Удалил строки с пропущенным значением в жанре
df.dropna(subset=['Genre'], inplace = True)

##### Пропуски в столбце Genre также восстоновить возможности нет. Их также небольшое количество. Строки с пропусками в Genre также можно удалить. Это не повлияет на дальнейший анализ.

In [ ]:
# Удалил строки с пропущенным значением в году релиза
df.dropna(subset=['Year_of_Release'], inplace = True)

##### Пропусков в столбце Year_of_Release уже больше, некритичная потеря данных для анализа составялет 5%. В связи с тем, что восстановить эти строки не представляется возможным, решено их удалить

In [ ]:
# Заменил пропуски в значениях рейтинга на значение Unknown. Это действие понадобится в дальнейшем
# Написал его в этот блок, чтобы сохроанить структуру.
df['Rating'] = df['Rating'].fillna('unknown')

In [ ]:
# Выведем все дубликаты по столбцам name и platform
df[df.duplicated(subset = ['Name','Platform'], keep = False)]

In [ ]:
# Дубликатов небольшое количество, их можно удалить без критической потри данных
df = df.drop_duplicates(subset = ['Name','Platform'], keep = False).reset_index(drop=True)

In [ ]:
# Check
df[df.duplicated(subset = ['Name','Platform'], keep = False)]

In [ ]:
# Check
df.info()

##### Заполнил пропуски в столбце Rating значением unknown, восстановить пропуски истинным значением не представляется возможным. Пропуски в столбцах Critic_Score и User_Score решено оставить с свзи с тем, что заполнение их медианным и средним значением отразилось бы на дальнейшем исследовании и могло бы привести к некорректным результатам.

In [ ]:
# Привел названия столбцов к нижнему регистру
df.columns = df.columns.str.lower()

In [ ]:
# Check
df.head(5)

In [ ]:
df.info()

###### Видим в столбце year_of_relaese тип float64, необходимо привести его к Int64. Год - целое число

In [ ]:
# Привел значения в столбце year_of_release к Int64
df['year_of_release'] = df['year_of_release'].astype('Int64')

In [ ]:
# Check
df.info()

In [ ]:
# Вывел уникальные значения оценки игры  гроками
df['user_score'].unique()

In [ ]:
# Выявил рейтинг tbd. Предположил, что это to be determined, то есть рейтиг формируется. 
# Такое возможно, если это новый релиз. Решил подробнее изучить строки с данной оценкой игроков
df.query('user_score == "tbd"')

In [ ]:
# Предположение не подтвердилось. Рейтинг tbd имеют релизы как новые, так и старые. 
# Также это может быть связанос тем, что продажи данных игр были слишком малы для формирования рейтинга.
# Решил заменить значения tbd в столбце  user_score на NaN. Что значит, что в данный момент рейтинг отсутствует.
df['user_score'] = df['user_score'].replace('tbd', np.NaN, regex=True)

In [ ]:
# Check
df['user_score'].unique()

In [ ]:
# Привел значения столбца user_score к типу float
df['user_score'] = df['user_score'].astype('float64')

###### Привел значения в столбце user_score к типу float64, так как оценкци выражены в числах с плавающей точкой.

In [ ]:
# Добавил столбец общих продах, который равен сумме продаж по всем регионам
df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales']

In [ ]:
# Check
df.head(5)

In [ ]:
df.info()

## Шаг 3. Исследовательский анализ данных

In [ ]:
# проверил число релизов по годам
releases_count = df.groupby('year_of_release')['name'].count()
releases_count.plot(kind='bar', grid=True, figsize = (15,5), title='Число релизов в год');

In [ ]:
# Оценил суммарные продажи по всем платформам
platform_total_sales = df.groupby(['platform'])['total_sales'].sum()
platform_total_sales.sort_values(ascending=False)

##### Максимальные продажи ха весь периожд наблюдения у платформ PS2 и Xbox360. Оценим динамику продаж на этих платформах по годам.

In [ ]:
# Создал датафрейм с релизами только на PS2 и XBox360
max_total_sales_platform = df.loc[(df['platform'] == "PS3") | (df['platform'] == "X360")]
# Оценил динамику продаж на этих платформах по годам
max_total_sales_platform_pivot = max_total_sales_platform.pivot_table(index=['year_of_release'],
                                                  columns='platform',
                                                  values='total_sales',
                                                  aggfunc='sum')
max_total_sales_platform_pivot.plot(kind='line',grid=True,figsize=(15, 7));

##### По данному графику можно оценить срок жизни платформы. В предыдущем поколении  он состовил 8 лет активных продаж новых релизов.

##### В связи с тем, что наш анализ направлен на оценку нынешней популярности платформ и прогнозе  их популярности в будущем, решил в дальнейшем анализе оставить релизы с 2014 по 2016 год

In [ ]:
# взял срез вктуальных релизов, чтобы сократить число платворим, на которые уже не выходят новые игры 
# для оценки прадаж по платформам в дальнейшем
actual_release = df.loc[(df['year_of_release']>=2014) & (df['year_of_release']<=2016)]
# оценил продажи по платформам, на которых выходят актуальные игры
artual_release_pivot = actual_release.pivot_table(index=['year_of_release'],
                                                  columns='platform',
                                                  values='total_sales',
                                                  aggfunc='sum')
# постотоил график
artual_release_pivot.plot(kind='line',grid=True,figsize=(15, 7), title = 'Динамика продаж актуальных релизов за 2015-2016');

#### По данному графику можно судить о том, что новые релизы выходят на PS4 и XOne, они являются актуальными платформами по которым в дальшейшем будем делать анализ. На текущий период отмечается спад продаж по всем платформам в течение последнего года.

In [ ]:
actual_release.boxplot(column='total_sales', by='platform', figsize=(25,7));

##### По данному графику можно судить о том, что наибольшие продажи актуальных релизов на платформах Nintendo 3DS, XBoxOne и PS4

In [ ]:
actual_release_1 = actual_release.loc[actual_release['total_sales']<=2].boxplot(
    column='total_sales', 
    by='platform', 
    figsize=(25,7))

##### Тот же график диаграмма размаха со сниженным значением total_sales

In [ ]:
actual_release

In [ ]:
# Создал DataFrame actual_platform  в котором отражаются актуальные релизы на PS4 и XOne за 2014-2016 год
actual_platform = actual_release.loc[(actual_release['platform'] == 'PS4') | (actual_release['platform'] == 'XOne')]

In [ ]:
# Создал DataFrame actaul_PS4  в котором отражены релизы на PS4 с 2014 по 2016
actual_PS4 = actual_platform.loc[actual_platform['platform'] == 'PS4']

In [ ]:
# Создал DataFrame actaul_XOne  в котором отражены релизы на XBoxOne с 2014 по 2016
actual_XOne = actual_platform.loc[actual_platform['platform'] == 'XOne']

In [ ]:
# Построил диаграмму рассеяния влияния оценки критиков на продажи актуальных релизов на PS4
actual_PS4.plot(x='critic_score', y='total_sales', kind='scatter', alpha=0.5, grid=True, 
                title='Влияние оценки критиков на продажи актуальных релизов на PS4');
# Посчитал коэффициент Пирсона влияния оценки критиков на продажи актуальных релизов на PS4
print('Коэффициент Пирсона =', actual_PS4['critic_score'].corr(actual_PS4['total_sales']))

##### Существует слабая связь между оценками критиков и общими продажами новых релизов на PS4. В некоторых случаях релизы получившие более высокие оценки лучше продаются

In [ ]:
# Построил диаграмму рассеяния влияния оценки игроков на продажи актуальных релизов на PS4
actual_PS4.plot(x='user_score', y='total_sales', kind='scatter', alpha=0.5, grid=True,
               title='Влияние оценки игроков на продажи актуальных релизов на PS4');
# Посчитал коэффициент Пирсона влияния оценки игроков на продажи актуальных релизов на PS4
print('Коэффициент Пирсона =',actual_PS4['user_score'].corr(actual_PS4['total_sales']))

##### Связь между оценками игроков и общими продажами новых релизов на PS4 отсутствует. Что говорит о том, новый релиз даже с низкими оценками игроков может иметь коммерческий успех 

In [ ]:
# Построил диаграмму рассеяния влияния оценки критиков на продажи актуальных релизов на XBoxOne
actual_XOne.plot(x='critic_score', y='total_sales', kind='scatter', alpha=0.5, grid=True,
                title='Влияние оценки критиков на продажи актуальных релизов на XBoxOne');
# Посчитал коэффициент Пирсона влияния оценки критиков на продажи актуальных релизов на XBoxOne
print('Коэффициент Пирсона =', actual_XOne['critic_score'].corr(actual_XOne['total_sales']))

##### Существует слабая связь между оценками критиков и общими продажами новых релизов на XBoxOne. Релизы получившие более высокие оценки в некоторых случаях лучше продаются, результат сопоставим с полученным результатом на PS4¶

In [ ]:
# Построил диаграмму рассеяния влияния оценки игроков на продажи актуальных релизов на XBoxOne
actual_XOne.plot(x='user_score', y='total_sales', kind='scatter', alpha=0.5, grid=True,
                title='Влияние оценки игроков на продажи актуальных релизов на XBoxOne');
# Посчитал коэффициент Пирсона влияния оценки игроков на продажи актуальных релизов на XBoxOne
print('Коэффициент Пирсона =',actual_XOne['user_score'].corr(actual_XOne['total_sales']))

##### Связь между оценками игроков и общими продажами новых релизов на XBoxOne отсутствует. Релизы получившие низкие оценки игроков могут иметь коммерческий успех.

In [ ]:
# Оценил самые прибыльные жанры всех релизов
genre_sales = df.groupby('genre')['total_sales'].sum()
genre_sales_sorted = genre_sales.sort_values(ascending=True)
genre_sales_sorted.plot(kind = 'barh', figsize = (19,6), fontsize = 16, 
                        title='Прибыльность жанров за все время продаж');

#####  Самыми прибыльными жанрами за все время наблюдения являются Action, Sports, Shooter, Role_Playing

In [ ]:
# Оценил самые прибыльные жанры на актуальных платформах 
acrual_genre_sales = actual_platform.groupby('genre')['total_sales'].sum()
acrual_genre_sales_sorted = acrual_genre_sales.sort_values(ascending=True)
acrual_genre_sales_sorted.plot(kind = 'barh', figsize = (19,6), fontsize = 16, 
                               title='Прибыльность жанров на актуальных платформах');

In [ ]:
actual_release.boxplot(column='total_sales', by='genre', figsize=(25,7));

In [ ]:
actual_release.boxplot(column='total_sales', by='genre', figsize=(25,7));
ylim=(0,2)

#####  На данной диаграмме размаха можно оценить продажи актуальных релизов в разбтвке по жанрам. Самым прибыльным жанром среди актульных релизов является жанр Shooter, у релизов этого жанра наблюдается высокий показатель медианы и межквартильного размаха продаж. Жано Sports также показывает хорошие показатели медианы продаж, но новые релизы на платформах, определенных, как популярные, имеют меньший коммерческий успех в отличии от жанра Action. 

In [ ]:
# Создал срез актуальных релизов жанра Shooter
actual_shooter = actual_release.loc[actual_release['genre'] == 'Shooter']
# Вывел более детальное числовое описание общих продаж новых релизов жанра Shooter
actual_shooter['total_sales'].describe()

In [ ]:
# Создал срез актуальных релизов жанра Action
actual_action = actual_release.loc[actual_release['genre'] == 'Action']
# Вывел более детальное числовое описание общих продаж новых релизов жанра Action
actual_action['total_sales'].describe()

In [ ]:
# Создал срез актуальных релизов жанра Sports
actual_sports = actual_release.loc[actual_release['genre'] == 'Sports']
# Вывел более детальное числовое описание общих продаж новых релизов жанра Sports
actual_sports['total_sales'].describe()

##### Самым прибыльным жанром на актуальных платформах является Shooter. Action показал себя более прибыльным чем Sports.

In [ ]:
# Отдельно решил оценить прибыльные жанры на PS4 
actual_PS4_genre = actual_PS4.groupby('genre')['total_sales'].sum()
actual_PS4_genre_sorted = actual_PS4_genre.sort_values(ascending=True)
actual_PS4_genre_sorted.plot(kind = 'barh', figsize = (19,6), fontsize = 16, title='Прибыльность жанров на PS4');

##### На PS4 наиболее прибыльным оказался жанр Action, на втором месте жанр Shooterб на третьем Sports, на четвертом также остается Role_Playing

In [ ]:
# Также отдельная оценка прибыльных жанров на XBoxOne 
actual_XOne_genre = actual_XOne.groupby('genre')['total_sales'].sum()
actual_XOne_genre_sorted = actual_XOne_genre.sort_values(ascending=True)
actual_XOne_genre_sorted.plot(kind = 'barh', figsize = (19,6), fontsize = 16, title='Прибыльность жанров на XBoxOne');

##### На XBoxOne самым прибыльным жанром оказался Shooter. на втором месте Action, Sports и Role_Playing не изменили своих позиций

### Портрет пользователя в Северной Америке

In [ ]:
# Оценил на каких платформах лучше всего продаются новые релизы в Северной Америке
na_top_platform = actual_release.groupby('platform')['na_sales'].sum().sort_values(ascending=True)
na_top_platform.plot(kind = 'barh', figsize = (19,6), fontsize = 16, title='Актуальность платформ в Северной Америке');

##### Самыми популярными платформами для покупки новых релизов в Северной Америке являются PS4 и XBoxOne. Между основными платформами наблюдается небольшой отрыв. Лидирует PS4

In [ ]:
na_top_platform = actual_release.groupby('genre')['na_sales'].sum().sort_values(ascending=True)
na_top_platform.plot(kind = 'barh', figsize = (19,6), fontsize = 16, title='Актуальность жанров в Северной Америке');

##### Самыми популярными жанрами в Северной Америке являются Shooter, Action, Sports, Role-Playing

In [ ]:
na_rating = actual_release.groupby('rating')['na_sales'].agg(['sum']).reset_index()
na_rating = na_rating.sort_values(by='sum',ascending=False)
na_rating = na_rating.set_index('rating')
na_rating.plot(kind='pie', y='sum', autopct='%1.0f%%', legend=False, figsize=(7,7), title='Рейтинг релизов ESRB');

##### Наибольший коммерческий успех у релизов с рейтингом М(для лиц старше 17 лет)

#### Таким образом в Северной Америке наибольшую популярность имеют платформы PS4 и XBoxOne. Наиболее популярными жанрами(топ-3) являются: Shooter,  Action, Sport. Наиболее популярны релизы с рейтингом М(для лиц старше 17 лет)

### Портрет пользователя в Европе

In [ ]:
# Оценил на каких платформах лучше всего продаются новые релизы в Европе
eu_top_platform = actual_release.groupby('platform')['eu_sales'].sum().sort_values(ascending=True)
eu_top_platform.plot(kind = 'barh', figsize = (19,6), fontsize = 16, title='Актуальность платформ в Европе');

##### Основными платформами для покупки новых релизов являются PS4 и XBoxOne, причем PS4 лидируется с большим отрывом.

In [ ]:
eu_top_platform = actual_release.groupby('genre')['eu_sales'].sum().sort_values(ascending=True)
eu_top_platform.plot(kind = 'barh', figsize = (19,6), fontsize = 16, title='Актуальность жанров в Европе');

##### В Европе чаще покупают игры жанра Action, чем Shooter. Эти жанры являются лидирующими. На третьем месте Sports, за ним Role_Playing

In [ ]:
eu_rating = actual_release.groupby('rating')['eu_sales'].agg(['sum']).reset_index()
eu_rating = eu_rating.sort_values(by='sum',ascending=False)
eu_rating = eu_rating.set_index('rating')
eu_rating.plot(kind='pie', y='sum', autopct='%1.0f%%', legend=False, figsize=(7,7), title='Рейтинг релизов ESRB');

##### В Европе также наблюдается наибольший коммерческий успех у релизов с рейтингом М(для лиц старше 17 лет)

#### В Европе наибольший коммерческий успех имеют платформы PS4 и XBoxOne. Но PS4 лидирует с гораздо большим отрывом. Топ - 3 популярными жанрами являются: Action, Shooter, Sports. Коммерческий успех имеют релизы с рейтингом М(для лиц старше 17 лет)

### Портрет пользователя в Японии

In [ ]:
jp_top_platform = actual_release.groupby('platform')['jp_sales'].sum().sort_values(ascending=True)
jp_top_platform.plot(kind = 'barh', figsize = (19,6), fontsize = 16, title='Актуальность платформ в Японии');

##### Наибольшее число новых релизов в Японии покупают на Nintendo 3DS, эта платформа значительно лидирует в сравнении с PS4. В этом заключается основное отличие пользователей Европы и Северной Америки в сравнении с пользователями в Японии. На рынке Азии большой коммерческий успех имеют платформы NIntendo

In [ ]:
jp_top_platform = actual_release.groupby('genre')['jp_sales'].sum().sort_values(ascending=True)
jp_top_platform.plot(kind = 'barh', figsize = (19,6), fontsize = 16, title='Актуальность жанров в Японии');

##### На Японском рынке самым популярным на новых платформах является жанр Role_Playing, он незначительно превосходит второй по значимости жанр Action. На третьем месте жанр Fighting

In [ ]:
jp_rating = actual_release.groupby('rating')['jp_sales'].agg(['sum']).reset_index()
jp_rating = jp_rating.sort_values(by='sum',ascending=False)
jp_rating = jp_rating.set_index('rating')
jp_rating.plot(kind='pie', y='sum', autopct='%1.0f%%', legend=False, figsize=(7,7), title='Рейтинг релизов ESRB');

##### По продажам новых релизов у значительной части выявлен неопределенный рейтинг. В сравнении с пользователями Европы и Северной Америки релизы имеющие рейтинг М имеют меньший процент. Вероятно, это связано с популярностью платформы Nintendo, которая специализируется на семейном контенте для всех категорй пользователей.

#### В Японии самой аоаулярной платформос с большим отрывом является Nintendo 3DS.Топ-3 популярными жанрами являются: Role_Playing, Action, Fighting. В топе рейтинга корректно оценить показатели невозможно в виду популярности Nintendo в Японии. Релизы Nintendo имеют другой оценочный рейтинг CERO в нашем анализе эти релизы попали в категорию рейтинга unknown.

## Шаг 4. Проверка гипотез

### Гипотеза: cредние пользовательские рейтинги платформ Xbox One и PC одинаковые:
* H0: средние рейтинги по платформам одинаковые
* H1: средние рейтинги по платформам разные

In [ ]:
# Удалил пропуски в столбце user_score так как они помешали бы дальшейшему анализу
actual_release.dropna(subset=['user_score'], inplace=True)

In [ ]:
# Сделал срез строк с платформами XboxOne и PC
XOne = actual_release[actual_release['platform'] == 'XOne']
PC = actual_release[actual_release['platform'] == 'PC']
# Для анализа нам понадобятся колонки с рейтингами XboxOne и PC
XOne_rating = XOne['user_score']
PC_rating = PC['user_score']
# Уровень статистической значимости, если p-value окажется меньше него - отвергнем гипотезу
alpha = .05

results = st.ttest_ind(
    XOne_rating, 
    PC_rating)

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

In [ ]:
print('Средний пользовательский рейтинг релизов на XBox One равен:', XOne_rating.mean())
print('Средний пользовательский рейтинг релизов на PC равен:', PC_rating.mean())

* Гипотеза о равенстве средних значений пользовательского рейтинга новых релизов двух генеральных совокупностей платформ XBoxOne и PC не опровергнута. Средние рейтинги новых релизов по данным платформам одинаковые.

### Гипотеза: средние пользовательские рейтинги жанров Action и Sports разные.
* H0: средние рейтинги по жанрам одинаковые
* H1: средние рейтинги по жанрам разные

In [ ]:
# Сделал срез строк с жанрами Action и Sports
action = actual_release[actual_release['genre'] == 'Action']
sports = actual_release[actual_release['genre'] == 'Sports']
# Для анализа нам понадобятся колонки с пользовательскими рейтингами жанров Action и Sports
action_rating = action['user_score']
sports_rating = sports['user_score']
# Уровень статистической значимости, если p-value окажется меньше него - отвергнем гипотезу
alpha = .05 

results = st.ttest_ind(
    action_rating,
    sports_rating)

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу") 

In [ ]:
print('Средний пользовательский рейтинг релизов жанра Action равен:', action_rating.mean())
print('Средний пользовательский рейтинг релизов жанра Sports равен:', sports_rating.mean())

* Гипотеза о равенстве средних значений пользовательского рейтинга двух генеральных совокупностей данра Action и жанра Sports опровергнута. Средние рейтинги новых релизов в жанрах Action и Sports отличаются.

## Общий вывод

* Для анализа были взяты платформы с наибольшими продажами актуальных релизов за 2014 - 2016 год. Такими платформами оказались PS4 и XBoxOne. 
* Самыми прибыльными жанрами на актуальных платформах являются Shooter, Action.
* В Северной Америке и Европе также самыми популярными платформами являются PS4 и Xbox One, самыми популярными жарнами являются Action и Shooter. Популярной категорией продуктов в данных регионах является категоря M (для лиц старше 17 лет). Отдельно был рассмотрен портрет пользователя в Японии, в этом регионе самой популярной платформой является Nintendo 3DS, самым популярным жанром является Role-Playing. 
* В будущем году основную рекламную компанию для пользователей в Северной Америки и Европы рекомендовано проводить с платформами PS4 и XBoxOne, жанрами Action и Shooter, релизами категории М. В Японии рекомендовано проводить рекламную компанию с релизами на Nintendo 3DS, жанрами Role-Playing.